## Récupérer les coordonnées des centres de distribution

In [63]:
import pandas as pd

df = pd.read_excel("data/centres.xlsx")

In [64]:
names_centres = df["Nom"].tolist()
adresses_centres = df["Adresse"].tolist()

In [65]:
import requests

api_key = open("api_key", "r").read()

In [66]:
def get_coords(addresses):
    latitudes = []
    longitudes = []

    for address in addresses:
        response = requests.get(
            f"https://maps.googleapis.com/maps/api/geocode/json?address={address}&key={api_key}"
        )
        data = response.json()

        if data["status"] == "OK":
            location = data["results"][0]["geometry"]["location"]
            latitudes.append(location["lat"])
            longitudes.append(location["lng"])
        else:
            print(f"Geocoding failed for address {address}. Status:", data["status"])
    
    return latitudes, longitudes

In [68]:

latitudes, longitudes = get_coords(adresses_centres)

In [67]:
# for n, lat, long in zip(names_centres, latitudes, longitudes) :
#     print(    f"{n} : {lat}, {long}")

In [35]:
# Add the latitudes as new columns into the original excel file
df["Latitude"] = latitudes
df["Longitude"] = longitudes

In [36]:
df.to_excel("data/centres.xlsx", index=False)

## Récupérer les coordonnées des points de ramasse

In [70]:
pdr = pd.read_excel("data/points_de_ramasse.xlsx")
adresses_pdr = pdr["Adresse"].tolist()
# latitudes, longitudes = get_coords(adresses)

In [11]:
for lat, long in zip(latitudes, longitudes) :
    print(    f"{lat}, {long}")

43.6431035, 1.4160821
43.6319261, 1.4843222
43.5661466, 1.5171659
43.5864191, 1.5776498
43.644588, 1.3726111
43.6666046, 1.514161
43.5913663, 1.2960563


In [12]:
pdr["Latitude"] = latitudes
pdr["Longitude"] = longitudes
pdr.to_excel("data/centres.xlsx", sheet_name="Points de Ramasse", index=False)

## Créer la matrice de distances avec OSM

In [20]:
import pandas, requests, json
from tqdm import tqdm

centres = pd.read_excel("data/centres.xlsx")
pdrs = pd.read_excel("data/points_de_ramasse.xlsx")

lats = centres["Latitude"].to_list() + pdrs["Latitude"].to_list()
longs = centres["Longitude"].to_list() + pdrs["Longitude"].to_list()

[43.6391233, 43.3518938, 42.7911527, 43.8006816, 43.6467573, 43.2971934, 43.2103963, 43.5347748, 43.517648, 43.6848225, 43.5389381, 43.8381611, 43.376577, 43.599054, 43.66709549999999, 43.0846413, 43.47820900000001, 43.6218265, 43.567516, 43.5418696, 43.45654649999999, 43.4133038, 43.106199, 43.0834612, 43.60352750000001, 43.56513890000001, 43.5928408, 43.5771751, 43.40153, 43.6431035, 43.6319261, 43.5661466, 43.5864191, 43.644588, 43.6666046, 43.5913663]


In [21]:
url = 'http://router.project-osrm.org/route/v1/driving/'
matrix = []
for i in tqdm(range(len(lats))):
    line = []
    for j in range(len(longs)) :
        o1 = str(longs[i]) +',' + str(lats[i])
        o2 = str(longs[j]) +',' + str(lats[j])
        x = o1 + ';' + o2
        response = requests.get(url+x)
        data = json.loads(response.content)
        distance = data["routes"][0]["legs"][0]["distance"]
        line.append(distance)
    matrix.append(line)

100%|██████████| 36/36 [10:48<00:00, 18.02s/it]


In [23]:
names_centres = centres["Nom"].tolist() + pdrs["Nom"].tolist()
matrix_df = pd.DataFrame(matrix, columns=names_centres, index=names_centres)
matrix_df.to_excel("data/matrix.xlsx")
matrix_df.to_csv("data/matrix.csv")

In [7]:
def make_euclidean(matrix):
    corrections = 0
    for i in range(len(matrix)):
        for j in range(len(matrix)):
            if j == i:
                continue
            for k in range(len(matrix)):
                if k == i or k == j:
                    continue
                if matrix.iloc[i, j] + matrix.iloc[j, k] < matrix.iloc[i, k]:
                    corrections += 1
                    matrix.iloc[i, k] = matrix.iloc[i, j] + matrix.iloc[j, k]
    print("Failures : ", corrections)

In [26]:
make_euclidean(matrix_df)
matrix_df.to_excel("data/euclidean_matrix_2.xlsx")
matrix_df.to_csv("data/euclidean_matrix_2.csv")

Failures :  786


## Créer la matrice de distances

In [72]:
import googlemaps

gmaps = googlemaps.Client(key=api_key)

In [77]:
from tqdm import tqdm

matrix = []
adresses = adresses_centres + adresses_pdr

for i, a in tqdm(enumerate(adresses), total=len(adresses)):
    matrix.append([])
    for j, b in enumerate(adresses):
        if i==j :
            matrix[i].append(0)
            continue
        
        results = gmaps.directions(a, b, alternatives=True,)
        results += gmaps.directions(a, b, alternatives=True, avoid="highways",)
        results += gmaps.directions(a, b, alternatives=True, avoid="tolls",)
        results += gmaps.directions(a, b, alternatives=True, avoid="tolls|highways",)
        dist = min(
            res["legs"][0]["distance"]["value"] for res in results
        )
        matrix[i].append(dist)

100%|██████████| 36/36 [08:51<00:00, 14.78s/it]


In [24]:
def make_euclidean(matrix):
    corrections = 0
    for i in range(len(matrix)):
        for j in range(len(matrix)):
            if j == i:
                continue
            for k in range(len(matrix)):
                if k == i or k == j:
                    continue
                if matrix.iloc[i, j] + matrix.iloc[j, k] < matrix.iloc[i, k]:
                    corrections += 1
                    matrix.iloc[i, k] = matrix.iloc[i, j] + matrix.iloc[j, k]
    print("Failures : ", corrections)

In [79]:
noms = df["Nom"].tolist() + pdr["Nom"].tolist()
mat = pd.DataFrame(matrix, columns=noms, index=noms)
mat.to_excel("data/non_euclidean_matrix.xlsx")
make_euclidean(mat)
mat.to_excel("data/euclidean_matrix.xlsx")

Failures :  610


## Convert xlsx files to csv for Hexaly

In [1]:
import pandas as pd

centres = pd.read_excel("data/centres.xlsx", index_col=0)
centres.to_csv("data/centres.csv", index=False)

In [ ]:
centres = pd.read_excel("data/centres_adjust.xlsx", index_col=0)
centres.to_csv("data/centres_adjust.csv", index=False)
centres = pd.read_excel("data/centres_adjust_or_keep.xlsx", index_col=0)
centres.to_csv("data/centres_adjust_or_keep.csv", index=False)
centres = pd.read_excel("data/centres_keep.xlsx", index_col=0)
centres.to_csv("data/centres_keep.csv", index=False)

In [2]:
vehicles = pd.read_excel("data/vehicules.xlsx")
vehicles.to_csv("data/vehicules.csv", index=False)

In [3]:
matrix = pd.read_excel("data/euclidean_matrix.xlsx")
matrix.to_csv("data/euclidean_matrix.csv", index=False)


In [4]:
points_de_ramasse = pd.read_excel("data/points_de_ramasse.xlsx")
points_de_ramasse.to_csv("data/points_de_ramasse.csv", index=False)

In [5]:
duration_matrix = pd.read_excel("data/duration_matrix.xlsx")
duration_matrix.to_csv("data/duration_matrix.csv", index=False)

In [3]:
duration_matrix_w_traffic = pd.read_excel("data/duration_matrix_w_traffic.xlsx")
duration_matrix_w_traffic.to_csv("data/duration_matrix_w_traffic.csv", index=False)

## Read  the performance data on week assignments

In [3]:
import pandas as pd
assignments = pd.read_csv("data/assignments.csv", index_col=0)
print(assignments)

                  assignments     week1     week2     total
0    (1, 2, 2, 1, 1, 1, 2, 2)  1721.290  1586.098  3307.388
1    (1, 2, 2, 2, 1, 1, 2, 1)  1724.907  1624.825  3349.732
2    (1, 2, 2, 1, 2, 2, 1, 2)  1652.529  1786.568  3439.097
3    (1, 1, 1, 1, 2, 2, 2, 2)  1692.834  1637.828  3330.662
4    (1, 1, 1, 1, 1, 2, 2, 2)  1714.329  1564.158  3278.487
..                        ...       ...       ...       ...
123  (1, 2, 2, 1, 2, 2, 2, 1)  1748.011  1715.253  3463.264
124  (1, 1, 1, 2, 2, 2, 1, 1)  1744.964  1655.453  3400.417
125  (1, 2, 1, 1, 1, 1, 1, 1)  2043.330  1537.133  3580.463
126  (1, 2, 1, 1, 1, 1, 2, 2)  1913.007  1558.053  3471.060
127  (1, 2, 2, 2, 1, 2, 1, 2)  1596.512  1822.740  3419.252

[128 rows x 4 columns]


In [4]:
assignments.sort_values(by="assignments", inplace=True)

In [6]:
print(assignments)

                  assignments     week1     week2     total
41   (1, 1, 1, 1, 1, 1, 1, 1)  1892.076  1539.599  3431.675
5    (1, 1, 1, 1, 1, 1, 1, 2)  1903.656  1360.742  3264.398
98   (1, 1, 1, 1, 1, 1, 2, 1)  1922.197  1493.128  3415.325
121  (1, 1, 1, 1, 1, 1, 2, 2)  1812.830  1496.939  3309.769
101  (1, 1, 1, 1, 1, 2, 1, 1)  1799.419  1489.704  3289.123
..                        ...       ...       ...       ...
8    (1, 2, 2, 2, 2, 1, 2, 2)  1807.835  1565.279  3373.114
67   (1, 2, 2, 2, 2, 2, 1, 1)  1718.547  1671.253  3389.800
71   (1, 2, 2, 2, 2, 2, 1, 2)  1622.120  1767.746  3389.866
15   (1, 2, 2, 2, 2, 2, 2, 1)  1599.843  1797.032  3396.875
110  (1, 2, 2, 2, 2, 2, 2, 2)  1658.513  1919.684  3578.197

[128 rows x 4 columns]


## Créer la matrice de temps de trajets

In [2]:
import pandas as pd 
import requests, json
from tqdm import tqdm

centres = pd.read_excel("data/centres.xlsx")
pdrs = pd.read_excel("data/points_de_ramasse.xlsx")

lats = centres["Latitude"].to_list() + pdrs["Latitude"].to_list()
longs = centres["Longitude"].to_list() + pdrs["Longitude"].to_list()

In [6]:
url = 'http://router.project-osrm.org/route/v1/driving/'
matrix = []
for i in tqdm(range(len(lats))):
    line = []
    for j in range(len(longs)) :
        o1 = str(longs[i]) +',' + str(lats[i])
        o2 = str(longs[j]) +',' + str(lats[j])
        x = o1 + ';' + o2
        response = requests.get(url+x)
        data = json.loads(response.content)
        duration = data["routes"][0]["legs"][0]["duration"]
        line.append(duration)
    matrix.append(line)

100%|██████████| 35/35 [10:13<00:00, 17.52s/it]


In [8]:
names_centres = centres["Nom"].tolist() + pdrs["Nom"].tolist()
matrix_df = pd.DataFrame(matrix, columns=names_centres, index=names_centres)
matrix_df.to_excel("data/duration_matrix.xlsx")
matrix_df.to_csv("data/duration_matrix.csv")

## Créer la matrice de temps de trajet pour une heure spécifique (heure de pointe)

In [2]:
import googlemaps
import requests
import pandas as pd

api_key = open("api_key", "r").read()
gmaps = googlemaps.Client(key=api_key)

In [26]:
from tqdm import tqdm
from time import time
import datetime

matrix = []
df = pd.read_excel("data/centres.xlsx")
names_centres = df["Nom"].tolist()
adresses_centres = df["Adresse"].tolist()
pdr = pd.read_excel("data/points_de_ramasse.xlsx")
adresses_pdr = pdr["Adresse"].tolist()
adresses = adresses_centres + adresses_pdr

peak_hour = int(datetime.datetime(2024, 2, 13, 8).timestamp())

for i, a in tqdm(enumerate(adresses), total=len(adresses)):
    matrix.append([])
    for j, b in enumerate(adresses):
        if i==j :
            matrix[i].append(0)
            continue
        
        results = gmaps.directions(a, b, alternatives=True, departure_time=peak_hour)
        duration = min(res["legs"][0]["duration_in_traffic"]["value"] for res in results)

        matrix[i].append(duration)

100%|██████████| 35/35 [05:27<00:00,  9.34s/it]


In [28]:
noms = df["Nom"].tolist() + pdr["Nom"].tolist()
mat = pd.DataFrame(matrix, columns=noms, index=noms)
mat.to_excel("data/duration_matrix_w_traffic.xlsx")